# Итоговый проект (часть 1: краулер) #
### Елизавета Клыкова, БКЛ181 ###
**Описание:** программа собирает тексты работ, опубликованных на международном ресурсе https://archiveofourown.org (все работы относятся к фандому "Mr. Robot (TV Show)"). Тексты обрабатываются морфологическими парсерами, данные сохраняются в базу SQLite. Для русского и английского языков создаются векторные модели, которые затем сравниваются по результатам разных тестов. Английской модели можно предложить текст какой-нибудь  работы, и она заменит в нем полнозначные слова на ближайшие синонимы (реализовано только на английских текстах, т.к. там меньше морфологии). Наконец, построены вордклауды наиболее частотных полнозначных слов русского и английского языков (чтобы выяснить, насколько похожи или различаются по содержанию работы на разных языках).

**Используемые инструменты:** краулеры, SQLite, морфологические парсеры, Word2vec, визуализации, dataframe.

Сначала включим проверку на PEP-8 и импортируем все необходимые для работы модули.

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import random
import re
import requests
import sqlite3
import time
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
session = requests.session()

### Пункт 1. Получение данных (краулеры) ###
Функция *parse_main_page* получает информацию о работах на главной странице фандома (по 20 работ на странице, сортировка по новизне). Многие элементы не являются обязательными, поэтому для их получения добавляем try-except. В качестве уникального fanfic_id будем использовать число из ссылки на полный текст работы.

In [3]:
def parse_main_page(f):
    ff = {}
    # к ссылке добавляем '?view_full_work=true',
    # чтобы не обходить главы по одной
    ff['full_link'] = 'https://archiveofourown.org' + f.find(
        'a').attrs['href'] + '?view_full_work=true'
    ff['fanfic_id'] = (re.search('([0-9])+', ff['full_link'])).group()
    ff['title'] = f.find('a').text

    # для авторов с закрытым аккаунтом
    try:
        ff['author'] = f.find('a', {'rel': 'author'}).text
        ff['author_link'] = 'https://archiveofourown.org' + f.find(
            'a', {'rel': 'author'}).attrs['href']
    except AttributeError:
        ff['author'] = 'restricted'
        ff['author_link'] = 'restricted'

    fandoms = []
    for fd in f.find_all('h5', {'class': 'fandoms'}):
        fandoms.append(fd.find('a').text)
    ff['fandoms'] = ';'.join(fandoms)
    ff['rating'] = f.find('span', {'class': 'rating'}).attrs['title']
    ff['category'] = f.find(
        'span', {'class': 'category'}).attrs['title']
    ff['status'] = f.find('span', {'class': 'iswip'}).attrs['title']
    ff['language'] = f.find('dd', {'class': 'language'}).text
    ff['chapters'] = f.find('dd', {'class': 'chapters'}).text
    try:
        ff['summary'] = f.find('blockquote').text
    except AttributeError:
        ff['summary'] = ''

    # кол-во слов иногда не отображается, поэтому проверка
    try:
        words = f.find('dd', {'class': 'words'}).text
        n_words = int(words.replace(',', ''))
    except ValueError:
        n_words = 0
    ff['words'] = n_words

    # предупреждения
    try:
        warnings = []
        for w in f.find_all('li', {'class': 'warnings'}):
            warning = w.find('a').text
            warnings.append(warning)
        ff['warnings'] = ';'.join(warnings)
    except AttributeError:
        ff['warnings'] = 'Not Stated'

    # пэйринги
    try:
        pairings = []
        for p in f.find_all('li', {'class': 'relationships'}):
            pairing = p.find('a').text
            pairings.append(pairing)
        ff['pairings'] = ';'.join(pairings)
    except AttributeError:
        ff['pairings'] = 'Not Stated'

    # персонажи
    try:
        characters = []
        for c in f.find_all('li', {'class': 'characters'}):
            character = c.find('a').text
            characters.append(character)
        ff['characters'] = ';'.join(characters)
    except AttributeError:
        ff['characters'] = 'Not Stated'

    # дополнительные теги
    try:
        tags = []
        for t in f.find_all('li', {'class': 'freeforms'}):
            tag = t.find('a').text
            tags.append(tag)
        ff['tags'] = ';'.join(tags)
    except AttributeError:
        ff['tags'] = 'Not Stated'

    # статистика
    try:
        comments = f.find('dd', {'class': 'comments'}).text
        ff['comments'] = int(comments.replace(',', ''))
    except (AttributeError, ValueError):
        ff['comments'] = 0
    try:
        bookmarks = f.find('dd', {'class': 'bookmarks'}).text
        ff['bookmarks'] = int(bookmarks.replace(',', ''))
    except (AttributeError, ValueError):
        ff['bookmarks'] = 0
    try:
        kudos = f.find('dd', {'class': 'kudos'}).text
        ff['kudos'] = int(kudos.replace(',', ''))
    except (AttributeError, ValueError):
        ff['kudos'] = 0
    try:
        hits = f.find('dd', {'class': 'hits'}).text
        ff['hits'] = int(hits.replace(',', ''))
    except (AttributeError, ValueError):
        ff['hits'] = 0

    return ff

Функция *parse_fanfic_page* получает полные тексты работ.

In [4]:
def parse_fanfic_page(ff):
    url_one = ff['full_link']
    # некоторые работы недоступны, поэтому проверка
    try:
        page = session.get(url_one).text
        soup = BeautifulSoup(page, 'html.parser')
        # получаем даты и полный текст
        ff['pub_date'] = soup.find('dd', {'class': 'published'}).text
        try:
            ff['upd_date'] = soup.find('dd', {'class': 'status'}).text
        except AttributeError:
            ff['upd_date'] = ff['pub_date']
        full_text = []
        for chapter in soup.find_all('div', {'class': 'userstuff'}):
            for p in chapter.find_all('p'):
                par = p.text
                full_text.append(par)
        ff['full_text'] = '\n'.join(full_text)
    except AttributeError:
        ff['pub_date'] = 'Unknown'
        ff['upd_date'] = 'Unknown'
        ff['full_text'] = 'Invalid URL'
    return ff

Функция *get_page* получает полную информацию обо всех работах на очередной странице.

In [5]:
def get_page(page_number):
    url = f'https://archiveofourown.org/tags/Mr*d*%20Robot%20(TV)/works?page={page_number}.html'
    page = session.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    fanfics = soup.find_all('li', {'class': 'work blurb group'})
    blocks = []
    for f in fanfics:
        try:
            blocks.append(parse_main_page(f))
        except Exception as e:
            print(e)
    result = []
    for b in blocks:
        try:
            res = parse_fanfic_page(b)
            result.append(res)
        except Exception as e:
            print(e)
    return result

2:80: E501 line too long (96 > 79 characters)


### Пункт 2. Запись в БД (SQLite) ###
Создадим базу данных с 5 таблицами. В основной таблице fanfics хранится общая информация и тексты работ, в таблицах warnings и tags - предупреждения и теги соответственно. Таблицы fanfic_to_warning и warning_to_tag связывают работы с предупреждениями и тегами связью один-ко-многим.

In [6]:
con = sqlite3.connect('eaklykova_final.db')
cur = con.cursor()

In [7]:
def create_database():
    cur.execute('''
    CREATE TABLE IF NOT EXISTS fanfics
    (id INTEGER PRIMARY KEY, fanfic_id text, title text, author text,
    author_link text, fandoms text, pairings text, characters text,
    rating text, category text, language text, status_ text, words int,
    chapters text, pub_date text, upd_date text, summary text,
    full_link text, full_text text, comments int, bookmarks int,
    kudos int, hits int)
    ''')

    cur.execute('''
    CREATE TABLE IF NOT EXISTS warnings
    (id INTEGER PRIMARY KEY, warning text)''')

    cur.execute('''
    CREATE TABLE IF NOT EXISTS fanfic_to_warning
    (id INTEGER PRIMARY KEY AUTOINCREMENT, id_fanfic int, id_warning int)
    ''')

    cur.execute('''
    CREATE TABLE IF NOT EXISTS tags
    (id INTEGER PRIMARY KEY, tag text)''')

    cur.execute('''
    CREATE TABLE IF NOT EXISTS fanfic_to_tag
    (id INTEGER PRIMARY KEY AUTOINCREMENT, id_fanfic int, id_tag int)
    ''')

    con.commit()
    con.close()

In [8]:
create_database()

Функция write_to_db принимает на вход список работ и записывает информацию о них в базу данных.

In [9]:
def write_to_db(blocks):
    errors = []
    for block in blocks:
        if block['fanfic_id'] not in seen_fanfics:
            seen_fanfics.add(block['fanfic_id'])
            tags = []
            warnings = []
            for tag in block['tags'].split(';'):
                if tag in db_tags:
                    tags.append(db_tags[tag])
                else:
                    db_tags[tag] = len(db_tags) + 1
                    tags.append(db_tags[tag])
                    cur.execute('INSERT INTO tags VALUES (?, ?)', (
                        len(db_tags), tag))
                    con.commit()
            for warning in block['warnings'].split(';'):
                if warning in db_warnings:
                    warnings.append(db_warnings[warning])
                else:
                    db_warnings[warning] = len(db_warnings) + 1
                    warnings.append(db_warnings[warning])
                    cur.execute('INSERT INTO warnings VALUES (?, ?)', (
                        len(db_warnings), warning))
                    con.commit()

            f_id = len(seen_fanfics)
            cur.execute('''
            INSERT INTO fanfics VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
            ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                        (f_id, block['fanfic_id'], block['title'],
                         block['author'], block['author_link'],
                         block['fandoms'], block['pairings'],
                         block['characters'], block['rating'],
                         block['category'], block['language'],
                         block['status'], block['words'], block['chapters'],
                         block['pub_date'], block['upd_date'],
                         block['summary'], block['full_link'],
                         block['full_text'], block['comments'],
                         block['bookmarks'], block['kudos'], block['hits'])
                        )

            tags = [(f_id, t) for t in tags]
            cur.executemany(
                'INSERT INTO fanfic_to_tag (id_fanfic, id_tag) VALUES (?, ?)',
                tags)
            con.commit()

            warnings = [(f_id, w) for w in warnings]
            cur.executemany(
                '''INSERT INTO fanfic_to_warning
                (id_fanfic, id_warning) VALUES (?, ?)''',
                warnings)
            con.commit()

        else:
            errors.append('Работа с id ' + block['fanfic_id']
                          + ' уже есть в базе')
    return errors

In [10]:
con = sqlite3.connect('eaklykova_final.db')
cur = con.cursor()

cur.execute('SELECT tag, id FROM tags')
db_tags = {}
for name, idx in cur.fetchall():
    db_tags[name] = idx

cur.execute('SELECT warning, id FROM warnings')
db_warnings = {}
for name, idx in cur.fetchall():
    db_warnings[name] = idx

cur.execute('SELECT fanfic_id FROM fanfics')
seen_fanfics = set(i[0] for i in cur.fetchall())

Функция run_all получает информацию со страниц с номерами от start_n до end_n.

In [11]:
def run_all(start_n, end_n):
    errors = []
    for i in tqdm(range(start_n, end_n+1)):
        try:
            err = write_to_db(get_page(i))
            errors.extend(err)
        except Exception as e:
            print(e)
    return errors

В выбранном фандоме около 950 работ, размещенных на 48 страницах. Выкачаем их все. Проходим по 5 страниц за раз, затем делаем перерыв 2.5-3 минуты (иначе ругается сайт).

In [12]:
a = 1
b = 5
for i in range(10):
    errors = run_all(a, b)
    a += 5
    # на последней итерации sleep не нужен
    if b != 48:
        time.sleep(random.randint(150, 180))
    # проверяем, что не превысили число страниц
    if b+5 > 48:
        b = 48
    else:
        b += 5

In [13]:
con.close()